# FVC 전처리

In [42]:
from ipywidgets import interact, interactive, fixed, interact_manual, link, HBox, Layout, interactive_output, VBox
import ipywidgets as widgets
import utils
import cv2
import glob, os
import numpy as np

from os.path import isdir, join
from os import listdir
import matplotlib.pyplot as plt

In [3]:
# closing
def closing_func(img, k_sz):
    kernel = np.ones((k_sz, k_sz), np.uint8)
    img_closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    result = cv2.subtract(img_closing, img)
    return result

def otsu_func(img):
    ret, th = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    print(f"++++++ OTSU vaule = {ret} +++++++")
    return ret, th

def opening_func(img):
    kernel = np.ones((15, 15), np.uint8)
    img_opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    result = cv2.subtract(img_opening, img)
    return result

def noise_reduction_func(img, kernel_r = 3, threshold=220):
    kernel = np.ones((kernel_r, kernel_r), np.float32) / (kernel_r**2)
    averaged_img = cv2.filter2D(img, -1, kernel)
    averaged_img[averaged_img<threshold] = 0
    averaged_img[averaged_img >= threshold] = 255

    return averaged_img

def rotate_image(finger, angle):
    # 이미지의 중심점을 구합니다.
    height, width = finger.shape[:2]
    center = (width / 2, height / 2)
    # 회전 변환 행렬을 생성합니다.
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    # 이미지를 회전시킵니다.
    rotated = cv2.warpAffine(finger, matrix, (width, height), borderValue=255)
    if angle == 0: return finger
    return rotated
"""
def circle_noise_removal_using_packing_density_func(img):
    from math import pi
    threshold = 0.1  # 0.12
    nlabels, img_labeled, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=8)

    for i in range(nlabels):
        if i < 1: continue  # 첫번째 노이즈가 자꾸 무시되어서 2를 1로 바꿔주었다.

        area = stats[i, cv2.CC_STAT_AREA]
        center_x = centroids[i, 0]
        center_y = centroids[i, 1]
        left = stats[i, cv2.CC_STAT_LEFT]
        top = stats[i, cv2.CC_STAT_TOP]
        width = stats[i, cv2.CC_STAT_WIDTH]
        height = stats[i, cv2.CC_STAT_HEIGHT]

        r = (width ** 2 + height ** 2) ** 0.5 / 2
        circle_area = pi * r * r
        key = area / circle_area

        if key > threshold: img[top: top + height, left: left + width] = 0
    return img
"""

'\ndef circle_noise_removal_using_packing_density_func(img):\n    from math import pi\n    threshold = 0.1  # 0.12\n    nlabels, img_labeled, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=8)\n\n    for i in range(nlabels):\n        if i < 1: continue  # 첫번째 노이즈가 자꾸 무시되어서 2를 1로 바꿔주었다.\n\n        area = stats[i, cv2.CC_STAT_AREA]\n        center_x = centroids[i, 0]\n        center_y = centroids[i, 1]\n        left = stats[i, cv2.CC_STAT_LEFT]\n        top = stats[i, cv2.CC_STAT_TOP]\n        width = stats[i, cv2.CC_STAT_WIDTH]\n        height = stats[i, cv2.CC_STAT_HEIGHT]\n\n        r = (width ** 2 + height ** 2) ** 0.5 / 2\n        circle_area = pi * r * r\n        key = area / circle_area\n\n        if key > threshold: img[top: top + height, left: left + width] = 0\n    return img\n'

## DataBase  설정

---
- FVC2000 Database2 사용 (256 × 364) - Genuin 쌍 2800개 (8C2 X 100)  
- FVC2000 Database4 사용 (240 x 320) - Genuin 쌍 2800개 (8C2 X 100)  
---
- FVC2002 Database1 사용 (384 × 374) - Genuin 쌍 2800개 (8C2 X 100)  
- FVC2002 Database3 사용 (300 x 300) - Genuin 쌍 2800개 (8C2 X 100)  
- FVC2002 Database4 사용 (268 x 384) - Genuin 쌍 2800개 (8C2 X 100)  
---
- FVC2004 Database2 사용 (328 x 364) - Genuin 쌍 2800개 (8C2 X 100)  
- FVC2004 Database4 사용 (288 x 284) - Genuin 쌍 2800개 (8C2 X 100)  
---
@ Total : 5600 560

In [56]:
FVC_YEAR = ['FVC2000',
            'FVC2002',
            'FVC2004']

ROOT_FVC_PATH = "/media/data2/jiwon/finger/data/FVC/Blind_Auth/raw/"

FVC00DB2A_DIR = join(ROOT_FVC_PATH, FVC_YEAR[0])

style = {'description_width': 'initial'}
input = widgets.Text(value=ROOT_FVC_PATH, 
             placeholder='Type something',
             description='**INPUT 1** || Your Pre-processing Root Dir (ex: /../../FVC/raw/):',
             style=style,
             disabled=False, layout=Layout(width='50%', height='30px'))

year = widgets.ToggleButtons(
    options= FVC_YEAR,
    value = FVC_YEAR[0],
    description='**INPUT 2** || DIR Year:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

# accordion = widgets.Accordion(children=[input, year], titles=('Root Path', 'FVC Year Dir'))
# display(accordion)


show = widgets.HTML(value=join(input.value, f'{year.value}/'), 
                    description='**RESULT 1** || Pre-processing Root Dir Path: ', 
                    style=style)
def change_show(_):
    show.value = join(input.value, f'{year.value}/')
    
input.observe(change_show)
year.observe(change_show)

display(widgets.VBox(children=[input, year, show]))

datas = sorted(os.listdir(show.value)) if isdir(show.value) else ['update plz']

subdir = widgets.ToggleButtons(
    options=datas,
    description='**INPUT 3** || DIR Subset:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)
show_sub = widgets.HTML(value=join(show.value, f'{subdir.value}/'),
                        description='**RESULT 2** || Pre-processing Sub Dir Path: ', 
                        style=style)
size = widgets.HTML(value=f'{len(sorted(listdir(show_sub.value)))}', 
                    description='**RESULT 2** || How many pictures are here : ', 
                    style=style)
index = widgets.BoundedIntText(value=686, 
                          min=0, 
                          max=len(listdir(join(show.value, f'{subdir.value}/')))-1, 
                          step=1,
                          description="Image Index :",
                          style=style)

def change_show_sub(_):
    subdir.options = datas = sorted(os.listdir(show.value)) if isdir(show.value) else ['update plz']
    show_sub.value = join(show.value, f'{subdir.value}/')
    size.value =f'{len(sorted(listdir(show_sub.value)))}'
    index.max = len(sorted(listdir(show_sub.value)))-1
    if index.value > index.max: index.value = 0 # index.max
    
    
show.observe(change_show_sub)
subdir.observe(change_show_sub)
size.observe(change_show_sub)

display(widgets.VBox(children=[subdir, show_sub, size]))
#####################
                        

def show_example(idx=0, blurType='None', otsuType=True, blk_sz=17, C=10,
                 br=15, pad = 0, thr_p=0.3, 
                 blurringmaskType=False, b_br=25, kbr=7, d_br=25, angle=0):
    
    # blk_sz = equl_args[0]
    # C = equl_args[1]
    
    file = join(show_sub.value, sorted(listdir(show_sub.value))[idx])
    img = cv2.imread(file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape[:2]
    otsu_value, otsu_img = otsu_func(gray)
    
    """ Function called by interact """
    if blurType == 'Median':
        gray = cv2.medianBlur(gray, 5)
    elif blurType == 'Guassian':
        gray = cv2.GaussianBlur(gray, (5, 5), 0)
    elif blurType == 'Bilateral': 
        gray = cv2.bilateralFilter(gray, 9, 75, 75)
            
    # _, thresh_cv = cv2.threshold(gray, th, 255, cv2.THRESH_BINARY) 
    adap_th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, blk_sz, C)
    gauss_blur = cv2.GaussianBlur(adap_th,(br,br),0)  #GaussianBlur br =9
    hist, bins = np.histogram(gauss_blur.flatten()*2, 256, [0,256]) #gauss_blur
    cdf = hist.cumsum()
    cdf_m = np.ma.masked_equal(cdf,0)
    cdf_m = ((cdf_m - cdf.min())*255/(cdf_m.max()-cdf.min()))
    cdf = np.ma.filled(cdf_m,0).astype('uint8')
    equalization = cdf[adap_th]
    
    padded_eq = cv2.copyMakeBorder(equalization, pad,pad,pad,pad, cv2.BORDER_CONSTANT, value=255)
    otsu_img = cv2.copyMakeBorder(otsu_img, pad,pad,pad,pad, cv2.BORDER_CONSTANT, value=255)
    
    sobel_img = otsu_img if otsuType else padded_eq
    if blurringmaskType:
        otsu_img = cv2.medianBlur(otsu_img, 55)
        sum_gm = cv2.GaussianBlur(otsu_img, (55, 55), 0)
    else:
        gx, gy = cv2.Sobel(sobel_img, cv2.CV_32F, 1, 0), cv2.Sobel(sobel_img, cv2.CV_32F, 0, 1)
        gx2, gy2 = gx**2, gy**2
        gm = np.sqrt(gx2 + gy2)
        sum_gm = cv2.boxFilter(gm, -1, (b_br, b_br), normalize = False)
        
    thr = sum_gm.max() * thr_p
    mask = cv2.threshold(sum_gm, thr, 255, cv2.THRESH_BINARY)[1].astype(np.uint8)
    if blurringmaskType: mask = cv2.bitwise_not(mask)
    
    # 구조화 요소 커널, 사각형 (3x3) 생성 ---①
    if d_br > 1: 
        d_k = cv2.getStructuringElement(cv2.MORPH_RECT, (d_br, d_br))
        mask = cv2.dilate(mask, d_k)
        
    e_k = cv2.getStructuringElement(cv2.MORPH_RECT, (kbr, kbr))
    erod = cv2.erode(mask, e_k)
    not_mask = cv2.bitwise_not(erod)
    # closed = closing_func(not_mask, k_sz=16)
    
    final_seg = np.bitwise_or(not_mask, padded_eq)
    if pad > 0: final_seg = final_seg[pad:-pad, pad:-pad]
    final_seg = rotate_image(final_seg, angle)
    
    utils.plot_images(img, adap_th, padded_eq, final_seg, cmap='gray')
    utils.plot_images(sum_gm, mask, erod, cmap='gray')        
    save_path = show_sub.value.replace('raw','seg')
    utils.save(save_path, file_name=sorted(listdir(show_sub.value))[idx], cv_img=final_seg, type="")
    
# Creating costume widget
threshSlider = widgets.FloatSlider(
    value = 0.30,
    min = 0.05,
    max = 0.95,
    step = 0.05,
    description = 'Threshold',
    continuous_update = False
)
CSlider = widgets.IntSlider(
    value = 11,
    min = 0,
    max = 29,
    step = 1,
    description = 'equalize C',
    continuous_update = False
)
paddingSlider = widgets.IntSlider(
    value = 0,
    min = 0,
    max = 20,
    step = 1,
    description = 'Padding size',
    continuous_update = False
)
bullkSlider = widgets.IntSlider(
    value = 13,
    min = 3,
    max = 35,
    step = 2,
    description = 'equalize Bulk',
    continuous_update = False
)
blurSlider = widgets.IntSlider(
    value = 5,
    min = 3,
    max = 23,
    step = 2,
    description = 'blur filter size',
    continuous_update = False
)
angleSlider = widgets.IntSlider(
    value = 0,
    min = -45,
    max = 45,
    step = 1,
    description = 'angle',
    continuous_update = False
)
erodBlurSlider = widgets.IntSlider(
    value = 5,
    min = 1,
    max = 61,
    step = 2,
    description = 'erod blur',
    continuous_update = False
)
dilBlurSlider = widgets.IntSlider(
    value = 1,
    min = 1,
    max = 51,
    step = 2,
    description = 'dilate blur',
    continuous_update = False
)
filterType = widgets.ToggleButtons(
    options=['None', 'Gaussian', 'Bilateral', 'Median'],
    disabled=False,
    value = 'Gaussian',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    # tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)
SimpleBlurmaskType = widgets.Checkbox(
    value=False,
    description='USE : Simple blur mask',
    disabled=False,
    indent=False
)
MorphfilterType = widgets.IntSlider(
    value = 13,
    min = 3,
    max = 55,
    step = 2,
    description = 'morph filter',
    continuous_update = False
)
out = interactive_output(show_example, {"idx"  : index,
        "thr_p"   : threshSlider,      "br"    : blurSlider,
        "blk_sz"  : bullkSlider,       "C"     : CSlider,
        "pad"     : paddingSlider,     "angle" : angleSlider,
        "kbr"     : erodBlurSlider,    "d_br"  : dilBlurSlider,
        "blurType": filterType,        "blurringmaskType":SimpleBlurmaskType,
        "b_br"    : MorphfilterType})

hbox0 = HBox([index, filterType])
hbox1 = HBox([blurSlider, angleSlider])
hbox2 = HBox([bullkSlider, CSlider])
hbox3 = HBox([paddingSlider, SimpleBlurmaskType])
hbox4 = HBox([threshSlider, MorphfilterType])
hbox5 = HBox([erodBlurSlider, dilBlurSlider])
ui = VBox([hbox0, hbox1, hbox2, hbox3, hbox4, hbox5])
display(ui, out)

Output()